In [144]:
#Create Spark Context
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [145]:
# import necessary libraries
from pyspark.sql.types import *
from pyspark.sql.window import Window
import pyspark.sql.functions as f
from pyspark.sql.functions import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Reading the data from files in HDFS

### Create input schema using StructType

In [146]:
fileSchema = StructType([StructField('year', IntegerType(),True),
StructField('month', StringType(),True),
StructField('day', IntegerType(),True),
StructField('weekday', StringType(),True),
StructField('hour', IntegerType(),True),
StructField('atm_status', StringType(),True),
StructField('atm_id', StringType(),True),
StructField('atm_manufacturer',  StringType(),True),
StructField('atm_location', StringType(),True),
StructField('atm_streetname', StringType(),True),
StructField('atm_street_number', IntegerType(),True),
StructField('atm_zipcode', IntegerType(),True),
StructField('atm_lat', DoubleType(), True),
StructField('atm_lon', DoubleType(), True),
StructField('currency', StringType(),True),
StructField('card_type', StringType(),True),
StructField('transaction_amount', IntegerType(),True),
StructField('service', StringType(),True),
StructField('message_code', StringType(),True),
StructField('message_text', StringType(),True),
StructField('weather_lat', DoubleType(), True),
StructField('weather_lon', DoubleType(), True),
StructField('weather_city_id', IntegerType(),True),
StructField('weather_city_name', StringType(),True),
StructField('temp', DoubleType(), True),
StructField('pressure', IntegerType(),True),
StructField('humidity', IntegerType(),True),
StructField('wind_speed', IntegerType(),True),
StructField('wind_deg', IntegerType(),True),
StructField('rain_3h', DoubleType(), True),
StructField('clouds_all', IntegerType(),True),
StructField('weather_id', IntegerType(),True),
StructField('weather_main', StringType(),True),
StructField('weather_description', StringType(),True)])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Read the data using the input schema created

In [147]:
df= spark.read.load('hdfs://ip-172-31-28-221.ap-northeast-1.compute.internal:8020/user/livy/data/part-m-00000',format='csv',  schema=fileSchema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [109]:
# print the schema
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- atm_status: string (nullable = true)
 |-- atm_id: string (nullable = true)
 |-- atm_manufacturer: string (nullable = true)
 |-- atm_location: string (nullable = true)
 |-- atm_streetname: string (nullable = true)
 |-- atm_street_number: integer (nullable = true)
 |-- atm_zipcode: integer (nullable = true)
 |-- atm_lat: double (nullable = true)
 |-- atm_lon: double (nullable = true)
 |-- currency: string (nullable = true)
 |-- card_type: string (nullable = true)
 |-- transaction_amount: integer (nullable = true)
 |-- service: string (nullable = true)
 |-- message_code: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- weather_lat: double (nullable = true)
 |-- weather_lon: double (nullable = true)
 |-- weather_city_id: integer (nullable = true)
 |-- weather_city_name: st

### verifying the data using count function

In [148]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

## Creation of Dimension tables(20%)

1. Location_DIM
2. DIM_CARD_TYPE
3. DIM_ATM
4. Date_DIM

In [149]:
# Create Location_DIM
df_location= df.select (['atm_location', 'atm_streetname', 'atm_street_number', 'atm_zipcode', 'atm_lat', 'atm_lon']).distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [154]:
# df_location.count()-109
# df.select('atm_location').distinct().count()-106
#df.select('atm_location','atm_streetname').distinct().count() -109

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

109

In [155]:
#create primarykey Location_id
windowSpec = Window.orderBy("atm_location")
df_location=df_location.withColumn("location_id",f.row_number().over(windowSpec))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [156]:
# reordering the columns
df_location= df_location.select('location_id', 'atm_location', 'atm_streetname','atm_street_number','atm_zipcode','atm_lat','atm_lon')
df_location.show()                       

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+------------------+-----------------+-----------+-------+-------+
|location_id|        atm_location|    atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|
+-----------+--------------------+------------------+-----------------+-----------+-------+-------+
|          1|             Aabybro|      ÃƒËœstergade|                6|       9440| 57.162|   9.73|
|          2|      Aalborg Hallen|      Europa Plads|                4|       9000| 57.044|  9.913|
|          3|Aalborg Storcente...|          Hobrovej|              452|       9200| 57.005|  9.876|
|          4|Aalborg Storcente...|          Hobrovej|              452|       9200| 57.005|  9.876|
|          5|         Aalborg Syd|          Hobrovej|              440|       9200| 57.005|  9.881|
|          6|           AalbÃƒÂ¦k|        Centralvej|                5|       9982| 57.593| 10.412|
|          7|              Aarhus|    SÃƒÂ¸nder Alle|               11|       8000| 56.153| 10.206|


In [157]:
#Create  DIM_CARD_TYPE
df_card_type= df.select('card_type').distinct()

# Create the primary key
windowcard = Window.orderBy("card_type")
df_card_type=df_card_type.withColumn("card_type_id",f.row_number().over(windowcard))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [158]:
#rearranging clumns
df_card_type=df_card_type.select('card_type_id','card_type')
df_card_type.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------------+
|card_type_id|           card_type|
+------------+--------------------+
|           1|              CIRRUS|
|           2|             Dankort|
|           3|     Dankort - on-us|
|           4|         HÃƒÂ¦vekort|
|           5| HÃƒÂ¦vekort - on-us|
|           6|             Maestro|
|           7|          MasterCard|
|           8|  Mastercard - on-us|
|           9|                VISA|
|          10|        Visa Dankort|
|          11|Visa Dankort - on-us|
|          12|            VisaPlus|
+------------+--------------------+

In [159]:
# create DIM_ATM
df_atm= df.select('atm_id', 'atm_manufacturer','atm_location', 'atm_streetname').distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [160]:
# As  Atm_number corresponds to the ATM_ID in the fact table we will rename the column
df_atm=df_atm.withColumnRenamed("atm_id","atm_number")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [161]:
# join the location_DIm with ATM_Im to get the locaion_ID
df_atm=df_atm.join(df_location, [df_atm.atm_location==df_location.atm_location ,\
                                 df_atm.atm_streetname==df_location.atm_streetname]\
                   ,'inner')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [162]:
windowatm = Window.orderBy("atm_number")
df_atm=df_atm.withColumn("atm_id_dim",f.row_number().over(windowatm))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [163]:
df_atm=df_atm.select('atm_id_dim', 'atm_number','atm_manufacturer', 'location_id')
df_atm.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+----------------+-----------+
|atm_id_dim|atm_number|atm_manufacturer|location_id|
+----------+----------+----------------+-----------+
|         1|         1|             NCR|         74|
|         2|        10|             NCR|         76|
|         3|       100|             NCR|         56|
|         4|       101|             NCR|         17|
|         5|       102|             NCR|          3|
|         6|       103| Diebold Nixdorf|        103|
|         7|       104|             NCR|         58|
|         8|       105| Diebold Nixdorf|         76|
|         9|       106|             NCR|         55|
|        10|       107| Diebold Nixdorf|         62|
|        11|       108|             NCR|         47|
|        12|       109| Diebold Nixdorf|          5|
|        13|        11|             NCR|         80|
|        14|       110| Diebold Nixdorf|         41|
|        15|       111| Diebold Nixdorf|          8|
|        16|       112| Diebold Nixdorf|      

In [165]:
#df.select('atm_id').distinct().count() -113
# df_atm.count()-113

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

113

In [166]:
# Create anew column for converting month to month number
df=df.withColumn("mon", from_unixtime(unix_timestamp(col("month"), 'MMM'),'MM') )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [167]:
# create a full date with yyyy-mm-dd hh:00:00 as as the monute and second values are not available
df=df.withColumn("full_date_time", f.to_timestamp(f.concat_ws(" ",f.concat_ws("-", "year", "mon", "day" ), "hour")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [168]:
# Create Date_DIM
df_date= df.select('full_date_time','year', 'month', 'day','hour', 'weekday' ).distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [169]:
windowdate = Window.orderBy("full_date_time")
df_date=df_date.withColumn("date_id",f.row_number().over(windowdate))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [170]:
df_date=df_date.select('date_id', 'full_date_time','year', 'month','day','hour','weekday')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [171]:
df_date.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+----+-------+---+----+-------+
|date_id|     full_date_time|year|  month|day|hour|weekday|
+-------+-------------------+----+-------+---+----+-------+
|      1|2017-01-01 00:00:00|2017|January|  1|   0| Sunday|
|      2|2017-01-01 01:00:00|2017|January|  1|   1| Sunday|
|      3|2017-01-01 02:00:00|2017|January|  1|   2| Sunday|
|      4|2017-01-01 03:00:00|2017|January|  1|   3| Sunday|
|      5|2017-01-01 04:00:00|2017|January|  1|   4| Sunday|
|      6|2017-01-01 05:00:00|2017|January|  1|   5| Sunday|
|      7|2017-01-01 06:00:00|2017|January|  1|   6| Sunday|
|      8|2017-01-01 07:00:00|2017|January|  1|   7| Sunday|
|      9|2017-01-01 08:00:00|2017|January|  1|   8| Sunday|
|     10|2017-01-01 09:00:00|2017|January|  1|   9| Sunday|
|     11|2017-01-01 10:00:00|2017|January|  1|  10| Sunday|
|     12|2017-01-01 11:00:00|2017|January|  1|  11| Sunday|
|     13|2017-01-01 12:00:00|2017|January|  1|  12| Sunday|
|     14|2017-01-01 13:00:00|2017|Januar

## Creation of Transaction Fact table(10%)

In [172]:
# join the df_card_type to get the card_type_id
df_fact=df.join(df_card_type, df_card_type.card_type==df.card_type,'inner').drop(("card_type"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [173]:
# join with df_date to get the date_id
df_fact=df_fact.join(df_date, df_date.full_date_time==df_fact.full_date_time,'inner').drop(*('full_date_time','year', 'month', 'day','hour', 'weekday', 'mon' ))



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [174]:
# join the df_atm to get the atm_id
df_fact=df_fact.join(df_atm, df_atm.atm_number==df_fact.atm_id,'inner'). drop(*('atm_manufacturer', 'atm_number', 'location_id', 'atm_id' ))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [175]:
# join with the location_dim to get the location_id
df_fact=df_fact.join(df_location, [df_fact.atm_location==df_location.atm_location , \
                                   df_fact.atm_streetname==df_location.atm_streetname],'inner')\
.drop(*('atm_location', 'atm_streetname', 'atm_street_number', 'atm_zipcode', 'atm_lat', 'atm_lon'))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [176]:
# create primary_key for the fact_table
windowdate = Window.orderBy('date_id')
df_fact=df_fact.withColumn("trans_id",f.row_number().over(windowdate))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [177]:
# As  Atm_number corresponds to the ATM_ID in the fact table
df_fact=df_fact.withColumnRenamed("location_id","weather_loc_id")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [178]:
df_fact=df_fact.select('trans_id','atm_id_dim',  'weather_loc_id', 'date_id', 'card_type_id', 'atm_status', 'currency',\
                      'service', 'transaction_amount', 'message_code', 'message_text', 'rain_3h',  'clouds_all', \
                      'weather_id', 'weather_main', 'weather_description')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [179]:
df_fact.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------+--------------+-------+------------+----------+--------+----------+------------------+------------+------------+-------+----------+----------+------------+--------------------+
|trans_id|atm_id_dim|weather_loc_id|date_id|card_type_id|atm_status|currency|   service|transaction_amount|message_code|message_text|rain_3h|clouds_all|weather_id|weather_main| weather_description|
+--------+----------+--------------+-------+------------+----------+--------+----------+------------------+------------+------------+-------+----------+----------+------------+--------------------+
|       1|        26|           103|      1|           9|  Inactive|     DKK|Withdrawal|              1891|        null|        null|   0.59|        92|       500|        Rain|          light rain|
|       2|        37|            48|      1|           9|  Inactive|     DKK|Withdrawal|              4166|        null|        null|    0.0|        75|       300|     Drizzle|light intensity d...|
|       3|

In [ ]:
## Loading dimension and fact tables to S3 bucket(5%)

In [180]:
df_fact.write.save("s3://etlprojectdipanwita/FACT_ATM_TRANS", format='csv', header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [181]:
df_location.write.save("s3://etlprojectdipanwita/DIM_LOCATION", format='csv', header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [182]:
df_atm.write.save("s3://etlprojectdipanwita/DIM_ATM", format='csv', header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [183]:
df_card_type.write.save("s3://etlprojectdipanwita/DIM_CARD_TYPE", format='csv', header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [184]:
df_date.write.save("s3://etlprojectdipanwita/DIM_DATE", format='csv', header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…